In [2]:
import pandas as pd
from google.colab import drive
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
import numpy as np

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
folder_path = "/content/drive/MyDrive/Translation"
os.chdir(folder_path)

In [ ]:
df = pd.read_excel('data.xlsx')
df.head()

,Sanskrit Word,English Meanings
0,Abhanavarana:,"['Screening the outshining Bragman', ' one of ..."
1,Abhasa:,"['Reflection', ' appearance', ' semblance', ' ..."
2,Abhasamatra:,['In name only.']
3,Abhasavada:,['Doctrine holding that all creation is reflec...
4,Abhati:,"['Shines', ' illumines.']"


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df['Sanskrit Tokenized'] = df['Sanskrit Word'].apply(nltk.word_tokenize)

In [ ]:
import spacy

In [ ]:
nlp_english = spacy.load("en_core_web_sm")

In [ ]:
def tokenize_english_sentence(english_sentence):
    english_doc = nlp_english(english_sentence)
    return [token.text for token in english_doc]

df['English Tokenized'] = df['English Meanings'].apply(tokenize_english_sentence)

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# Train or load Word2Vec model on the tokenized words for Sanskrit
word2vec_model_sanskrit = Word2Vec(sentences=df['Sanskrit Tokenized'], vector_size=100, window=5, min_count=1, workers=4)

# Train or load Word2Vec model on the tokenized words for English
word2vec_model_english = Word2Vec(sentences=df['English Tokenized'], vector_size=100, window=5, min_count=1, workers=4)

# Convert tokenized words to word embeddings for Sanskrit
df['Sanskrit Embeddings'] = df['Sanskrit Tokenized'].apply(lambda tokens: [word2vec_model_sanskrit.wv[word] for word in tokens])

# Convert tokenized words to word embeddings for English
df['English Embeddings'] = df['English Tokenized'].apply(lambda tokens: [word2vec_model_english.wv[word] for word in tokens])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Create a tokenizer for Sanskrit words
sanskrit_tokenizer = Tokenizer(oov_token='<OOV>')
sanskrit_tokenizer.fit_on_texts(df['Sanskrit Tokenized'])

# Create a tokenizer for English words
english_tokenizer = Tokenizer(oov_token='<OOV>')
english_tokenizer.fit_on_texts(df['English Tokenized'])

In [ ]:
# Convert tokenized Sanskrit sequences to index sequences
sanskrit_sequences = sanskrit_tokenizer.texts_to_sequences(df['Sanskrit Tokenized'])

# Convert tokenized English sequences to index sequences
english_sequences = english_tokenizer.texts_to_sequences(df['English Tokenized'])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the maximum sequence lengths (input and output)
max_sanskrit_length = 20
max_english_length = 25

# Pad or truncate the sequences to the maximum lengths
padded_sanskrit_sequences = pad_sequences(sanskrit_sequences, maxlen=max_sanskrit_length, padding='post', truncating='post')
padded_english_sequences = pad_sequences(english_sequences, maxlen=max_english_length, padding='post', truncating='post')

In [ ]:
# Input will be the padded Sanskrit sequences
encoder_input_data = padded_sanskrit_sequences

# Output will be the padded English sequences shifted by one time step (teacher forcing)
decoder_input_data = padded_english_sequences[:, :-1]
decoder_output_data = padded_english_sequences[:, 1:]

# The first time step of the output is not used (remove the first token from each sequence)
# The last token in each output sequence is <PAD> due to padding, and it won't have a corresponding input token
# To maintain consistency, the last token in each output sequence is also removed

In [ ]:
# Define the input shape for the model
input_shape = (max_sanskrit_length,)

# Define the size of the vocabulary for both Sanskrit and English
sanskrit_vocab_size = len(sanskrit_tokenizer.word_index) + 1
english_vocab_size = len(english_tokenizer.word_index) + 1

# Define the embedding dimension
embedding_dim = 100

# Encoder
encoder_input = Input(shape=input_shape)
encoder_embedding = Embedding(input_dim=sanskrit_vocab_size, output_dim=embedding_dim)(encoder_input)
encoder_lstm = LSTM(300)(encoder_embedding)  # Adjust the number of LSTM units as needed

# Decoder
decoder_input = Input(shape=(max_english_length - 1,))
decoder_embedding = Embedding(input_dim=english_vocab_size, output_dim=embedding_dim)(decoder_input)
decoder_lstm = LSTM(300, return_sequences=True)(decoder_embedding, initial_state=[encoder_lstm, encoder_lstm])
decoder_output = Dense(english_vocab_size, activation='softmax')(decoder_lstm)

# Model
model = Model(inputs=[encoder_input, decoder_input], outputs=decoder_output)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the summary of the model
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 20, 100)      248600      ['input_5[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, 24, 100)      384900      ['input_6[0][0]']                
                                                                                            

In [ ]:
# Train the model using provided data
model.fit([encoder_input_data, decoder_input_data], decoder_output_data, batch_size=32, epochs=80, validation_split=0.1)

Epoch 1/80
71/71 [==============================] - 14s 147ms/step - loss: 4.0134 - accuracy: 0.4880 - val_loss: 3.3168 - val_accuracy: 0.4931
Epoch 2/80
71/71 [==============================] - 5s 75ms/step - loss: 2.7936 - accuracy: 0.5511 - val_loss: 2.7840 - val_accuracy: 0.5567
Epoch 3/80
71/71 [==============================] - 3s 38ms/step - loss: 2.4122 - accuracy: 0.6278 - val_loss: 2.5335 - val_accuracy: 0.6324
Epoch 4/80
71/71 [==============================] - 1s 14ms/step - loss: 2.1700 - accuracy: 0.6743 - val_loss: 2.4033 - val_accuracy: 0.6508
Epoch 5/80
71/71 [==============================] - 2s 27ms/step - loss: 2.0437 - accuracy: 0.6902 - val_loss: 2.3637 - val_accuracy: 0.6632
Epoch 6/80
71/71 [==============================] - 1s 20ms/step - loss: 1.9725 - accuracy: 0.6965 - val_loss: 2.3530 - val_accuracy: 0.6667
Epoch 7/80
71/71 [==============================] - 2s 23ms/step - loss: 1.9179 - accuracy: 0.7011 - val_loss: 2.3409 - val_accuracy: 0.6682
Epoch 8/80


In [ ]:
# New Sanskrit word for translation
new_sanskrit_word = "Abhimani:"

# Tokenize and pad the new Sanskrit word
new_sanskrit_sequences = sanskrit_tokenizer.texts_to_sequences([new_sanskrit_word])
padded_new_sanskrit_sequences = pad_sequences(new_sanskrit_sequences, maxlen=max_sanskrit_length, padding='post', truncating='post')

# Perform inference for translation
encoder_input_for_translation = padded_new_sanskrit_sequences
decoder_input_for_translation = np.zeros((1, max_english_length - 1))  # Initialize decoder input with zeros
start_token_index = 1  # Default to index 1 as the start token index
if '<start>' in english_tokenizer.word_index:
    start_token_index = english_tokenizer.word_index['<start>']
decoder_input_for_translation[:, 0] = start_token_index  # Set the start token index
translated_word_sequence = []  # To store the translated word sequence

max_translation_length = 20  # Set a maximum translation length (adjust as needed)

for i in range(max_translation_length - 1):  # Loop until the maximum translation length
    predictions = model.predict([encoder_input_for_translation, decoder_input_for_translation])
    next_word_index = np.argmax(predictions[:, i, :], axis=-1)  # Get the index of the next word in the sequence
    translated_word_sequence.append(int(next_word_index))  # Convert numpy int to Python int
    decoder_input_for_translation[:, i+1] = next_word_index  # Update decoder input for the next time step

    if next_word_index == 0:  # Check if the predicted word is the padding token (index 0)
        break

# Convert the translated_word_sequence back to English words
translated_english_words = english_tokenizer.sequences_to_texts([translated_word_sequence])[0]

print("Sanskrit Word:", new_sanskrit_word)
print("Translated English Meaning:", translated_english_words)


1/1 [==============================] - 0s 31ms/step
Sanskrit Word: Abhimani:
Translated English Meaning: ' the state of being an experiencer or enjoyer . ' ] <OOV>
